In [1]:
import numpy as np

## Data set

In [2]:
x1 = np.array([ 0.        ,  3.33333333,  6.66666667, 10.        ,  0.        ,
        3.33333333,  6.66666667, 10.        ,  0.        ,  3.33333333,
        6.66666667, 10.        ,  0.        ,  3.33333333,  6.66666667,
       10.        ])
x2 = np.array([0.        , 0.        , 0.        , 0.        , 0.33333333,
       0.33333333, 0.33333333, 0.33333333, 0.66666667, 0.66666667,
       0.66666667, 0.66666667, 1.        , 1.        , 1.        ,
       1.        ])
y = np.array([ 1.22353179,  3.57574634,  5.89217837,  8.19889324,  1.96307308,
        4.57556858,  7.48364786,  9.2400334 ,  2.9048125 ,  5.74174653,
        8.19832673, 10.69424417,  4.15380028,  7.12275093,  8.8976015 ,
       11.76820779])

In [3]:
x1, x2, y

(array([ 0.        ,  3.33333333,  6.66666667, 10.        ,  0.        ,
         3.33333333,  6.66666667, 10.        ,  0.        ,  3.33333333,
         6.66666667, 10.        ,  0.        ,  3.33333333,  6.66666667,
        10.        ]),
 array([0.        , 0.        , 0.        , 0.        , 0.33333333,
        0.33333333, 0.33333333, 0.33333333, 0.66666667, 0.66666667,
        0.66666667, 0.66666667, 1.        , 1.        , 1.        ,
        1.        ]),
 array([ 1.22353179,  3.57574634,  5.89217837,  8.19889324,  1.96307308,
         4.57556858,  7.48364786,  9.2400334 ,  2.9048125 ,  5.74174653,
         8.19832673, 10.69424417,  4.15380028,  7.12275093,  8.8976015 ,
        11.76820779]))

## 1
a) Run a linear regression to find A, B1, and B2.  
b) Calculate SE(B1) and SE(B2)  
c) Calculate a prediction, y_hat, at x1=5, x2=.5 along with $SE(yhat) = \sqrt{RSS/(n-k-1)} / \sqrt{n}$

In [4]:
import pandas as pd
from sklearn.linear_model import LinearRegression

df = pd.DataFrame()
df['x1']=x1
df['x2']=x2
df['y']=y

# a
x = np.array(df[['x1','x2']])
reg = LinearRegression().fit(x, y)
A = reg.intercept_
print('A', round(A,4))
print('B1', round(reg.coef_[0],4))
print('B2', round(reg.coef_[1],4))

A 1.0325
B1 0.7382
B2 3.2575


In [5]:
#b
k = x.shape[1]
n = x.shape[0]
yhat=reg.predict(x)
E=y-yhat
SSE=sum(E**2)
SE=np.sqrt( SSE/(n-k-1) )
SE_B = SE/np.sqrt(sum((x-x.mean())**2))
print('SE_B1', round(SE_B[0],4))
print('SE_B2', round(SE_B[1],4))

SE_B1 0.0139
SE_B2 0.0265


In [6]:
# c
yhat_sample=reg.predict(np.array([[5,0.5]]))
print('yhat', yhat_sample)

RSS = sum((y-yhat_sample)**2)
SE_yhat = (np.sqrt(RSS/(n-k-1)))/np.sqrt(n)
print('SE_yhat', round(SE_yhat,4))

yhat [6.35213519]
SE_yhat 0.8362


## 2
a) Calculate m=100 bootstrap sample sets from the original set of 16 samples and, for each bootstrap sample set, estimate A_bs, B1_bs, and B2_bs by regression. (You'll have 100 estimates of each of A_bs, B1_bs, and B2_bs.)  
b) Compute SE(B1_bs) and SE(B2_bs) directly from the m=100 samples. Do they match SE(B1) and SE(B2) from part 1, above?  
c) For each model, A_bs, B1_bs, and B2_bs, calculate a prediction, y_hat_bs, at x1=5, x2=.5. From that set of predictions, compute SE(y_hat_bs). Does it match SE(y_hat) from part 1, above?  
  
  
Hint: Be sure the values of y, x1, and x2 stay aligned when you draw a sample from the original data. Ex: Randomly drawing the i'th sample would yield (y[i],x1[i],x2[i]) -- all with the same i.

In [7]:
# a
# x includes x1, x2
def bootstrap(x, y):
    i = np.random.randint(0,len(x),size=(len(x)))
    return x[i], y[i]

m=100
bootstrap_xs = []
bootstrap_ys = []
A_bs = []
B1_bs = []
B2_bs = []
model_bs = []
for _ in range(m):
    bootstrap_x, bootstrap_y = bootstrap(x, y)
    bootstrap_xs.append(bootstrap_x)
    bootstrap_ys.append(bootstrap_y)
    reg = LinearRegression().fit(bootstrap_x, bootstrap_y)
    model_bs.append(reg)
    A_bs.append(reg.intercept_)
    B1_bs.append(reg.coef_[0])
    B2_bs.append(reg.coef_[1])
    
# verify the output
print(len(bootstrap_xs), len(bootstrap_ys), len(A_bs), len(B1_bs), len(B2_bs))

100 100 100 100 100


In [8]:
# b
k = bootstrap_xs[0].shape[1]
n = bootstrap_xs[0].shape[0]

SE_B1 = []
SE_B2 = []
for i in range(m):
    E=bootstrap_ys[i]-model_bs[i].predict(bootstrap_xs[i])
    SSE=sum(E**2)
    SE=np.sqrt( SSE/(n-k-1) )
    SE_B = SE/np.sqrt(sum((bootstrap_xs[i]-bootstrap_xs[i].mean())**2))
    SE_B1.append(SE_B[0])
    SE_B2.append(SE_B[1])
print('length of SE_B1:', len(SE_B1), ', mean of SE_B1:', round(np.mean(SE_B1),4))
print('length of SE_B2:', len(SE_B2), ', mean of SE_B2:', round(np.mean(SE_B2),4))

length of SE_B1: 100 , mean of SE_B1: 0.0124
length of SE_B2: 100 , mean of SE_B2: 0.0236


- Part 1:

`SE_B1 0.0139`

`SE_B2 0.0265`

By comparing the SE_B1 and SE_B2 in the part1, it's very close to the result from bootstrap.

In [9]:
# c
y_hat_bs = [model_bs[i].predict(np.array([[5, 0.5]])) for i in range(m)]
SE_y_hat_bs = []
for i in range(m):
    RSS = sum((bootstrap_ys[i]-y_hat_bs[i])**2)
    SE_y_hat_bs.append((np.sqrt(RSS/(n-k-1)))/np.sqrt(n))
print('SE_y_hat_bs: ', round(np.mean(SE_y_hat_bs),4))

SE_y_hat_bs:  0.8313


- Part 1:

`SE_yhat 0.8362`

By comparing the SE_yhat in the part1, it's very close to the result from bootstrap.